<a href="https://colab.research.google.com/github/SaeedM75/Deep-learning/blob/main/HW2P2_Saeed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as ttf

import os
import os.path as osp

from tqdm import tqdm
from PIL import Image
from sklearn.metrics import roc_auc_score
import numpy as np
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!pip install wandb -q

     |████████████████████████████████| 59 kB 2.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73275 sha256=5018a549ff5bfd7f7849115d58d0fc0a564b06827d71d7e19b997e2516420d37
  Stored in directory: /root/.cache/pip/wheels/de/f7/d8/c3902cacb7e62cb611b1ad343d7cc07f42f7eb76ae3a52f3d1
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 181 kB 48.5 MB/s 
     |████████████████████████████████| 144 kB 58.4 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 


In [ ]:
import wandb
wandb.login()

from google.colab import drive  
drive.mount('/content/gdrive')


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Mounted at /content/gdrive


# TODOs
As you go, please read the code and keep an eye out for TODOs!

# Download Data

In [ ]:

!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"smoayedp","key":"42e16d25f2ea76fd2f8b490568d200ba"}') # Put your kaggle username & key here



!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c 11-785-s22-hw2p2-classification
!kaggle competitions download -c 11-785-s22-hw2p2-verification

!unzip -q 11-785-s22-hw2p2-classification.zip
!unzip -q 11-785-s22-hw2p2-verification.zip

!ls

 99% 2.33G/2.35G [00:10<00:00, 275MB/s]
100% 2.35G/2.35G [00:10<00:00, 241MB/s]
 97% 254M/263M [00:01<00:00, 227MB/s]
100% 263M/263M [00:01<00:00, 228MB/s]
11-785-s22-hw2p2-classification.zip   sample_data
11-785-s22-hw2p2-verification.zip     train_subset
classification			      verification
classification_sample_submission.csv  verification_sample_submission.csv
gdrive


# Hyperparameters

In [ ]:
"""
The well-accepted SGD batch_size & lr combination for CNN classification is 256 batch size for 0.1 learning rate.
When changing batch size for SGD, follow the linear scaling rule - halving batch size -> halve learning rate, etc.
This is less theoretically supported for Adam, but in my experience, it's a decent ballpark estimate.
"""
batch_size = 256
lr = 0.1

# batch_size = 128
# lr = 0.05
epochs = 20 # Just for the early submission. We'd want you to train like 50 epochs for your main submissions.

In [3]:
!git clone https://github.com/facebookresearch/ConvNeXt
!pip install timm==0.3.2 tensorboardX six

Cloning into 'ConvNeXt'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (231/231), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 231 (delta 118), reused 186 (delta 83), pack-reused 0
Receiving objects: 100% (231/231), 66.93 KiB | 1.52 MiB/s, done.
Resolving deltas: 100% (118/118), done.
     |████████████████████████████████| 244 kB 4.3 MB/s 
     |████████████████████████████████| 125 kB 31.8 MB/s 


In [4]:
!python /content/ConvNeXt/models/convnext.py

Traceback (most recent call last):
  File "/content/ConvNeXt/models/convnext.py", line 12, in <module>
    from timm.models.layers import trunc_normal_, DropPath
  File "/usr/local/lib/python3.7/dist-packages/timm/__init__.py", line 2, in <module>
    from .models import create_model, list_models, is_model, list_modules, model_entrypoint, \
  File "/usr/local/lib/python3.7/dist-packages/timm/models/__init__.py", line 1, in <module>
    from .cspnet import *
  File "/usr/local/lib/python3.7/dist-packages/timm/models/cspnet.py", line 20, in <module>
    from .helpers import build_model_with_cfg
  File "/usr/local/lib/python3.7/dist-packages/timm/models/helpers.py", line 17, in <module>
    from .layers import Conv2dSame, Linear
  File "/usr/local/lib/python3.7/dist-packages/timm/models/layers/__init__.py", line 7, in <module>
    from .cond_conv2d import CondConv2d, get_condconv_initializer
  File "/usr/local/lib/python3.7/dist-packages/timm/models/layers/cond_conv2d.py", line 16, in <mo

In [5]:
import torch
import torch.nn as nn
import math


class InvertedResidualBlock(nn.Module):
    """
    Intuitively, layers in MobileNet can be split into "feature mixing" 
    and "spatial mixing" layers. You can think of feature mixing as each pixel
    "thinking on its own" about its own featuers, and you can think of spatial
    mixing as pixels "talking with each other". Alternating these two builds
    up a CNN.

    In a bit more detail:

    - The purpose of the "feature mixing" layers is what you've already seen in 
    hw1p2. Remember, in hw1p2, we went from some low-level audio input to
    semantically rich representations of phonemes. Featuring mixing is simply a 
    linear layer (a weight matrix) that transforms simpler features into 
    something more advanced.

    - The purpose of the "spatial mixing" layers is to mix features from different
    spatial locations. You can't figure out a face by looking at each pixel on
    its own, right? So we need 3x3 convolutions to mix features from neighboring
    pixels to build up spatially larger features.
    """
    def __init__(self,
                 in_channels,
                 out_channels,
                 stride,
                 expand_ratio):
        super().__init__() # Just have to do this for all nn.Module classes

        # Can only do identity residual connection if input & output are the
        # same channel & spatial shape.
        if stride == 1 and in_channels == out_channels:
            self.do_identity = True
        else:
            self.do_identity = False
        
        # Expand Ratio is like 6, so hidden_dim >> in_channels
        hidden_dim = in_channels * expand_ratio

        """
        What is this doing? It's a 1x1 convolutional layer that drastically
        increases the # of channels (feature dimension). 1x1 means each pixel
        is thinking on its own, and increasing # of channels means the network
        is seeing if it can "see" more clearly in a higher dimensional space.

        Some patterns are just more obvious/separable in higher dimensions.

        Also, note that bias = False since BatchNorm2d has a bias term built-in.

        As you go, note the relationship between kernel_size and padding. As you
        covered in class, padding = kernel_size // 2 (kernel_size being odd) to
        make sure input & output spatial resolution is the same.
        """
        self.feature_mixing = nn.Sequential(
            nn.Conv2d(in_channels, hidden_dim, kernel_size= 1, stride = 1, padding = 0, bias = False),
            nn.BatchNorm2d(hidden_dim),
            nn.GELU()
            # TODO: Fill this in!
        )

        """
        What is this doing? Let's break it down.
        - kernel_size = 3 means neighboring pixels are talking with each other.
          This is different from feature mixing, where kernel_size = 1.

        - stride. Remember that we sometimes want to downsample spatially. 
          Downsampling is done to reduce # of pixels (less computation to do), 
          and also to increase receptive field (if a face was 32x32, and now
          it's 16x16, a 3x3 convolution covers more of the face, right?). It
          makes sense to put the downsampling in the spatial mixing portion
          since this layer is "in charge" of messing around spatially anyway.

          Note that most of the time, stride is 1. It's just the first block of
          every "stage" (layer \subsetof block \subsetof stage) that we have
          stride = 2.

        - groups = hidden_dim. Remember depthwise separable convolutions in 
          class? If not, it's fine. Usually, when we go from hidden_dim channels
          to hidden_dim channels, they're densely connected (like a linear 
          layer). So you can think of every pixel/grid in an input
          3 x 3 x hidden_dim block being connected to every single pixel/grid 
          in the output 3 x 3 x hidden_dim block.
          What groups = hidden_dim does is remove a lot of these connections.

          Now, each input 3 x 3 block/region is densely connected to the
          corresponding output 3 x 3 block/region. This happens for each of the
          hidden_dim input/output channel pairs independently.
          So we're not even mixing different channels together - we're only 
          mixing spatial neighborhoods. 
          
          Try to draw this out, or come to my (Jinhyung Park)'s OH if you want 
          a more in-depth explanation.
          https://towardsdatascience.com/a-basic-introduction-to-separable-convolutions-b99ec3102728
        """
        self.spatial_mixing = nn.Sequential(
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size= 3, padding= 1, stride = stride, groups= hidden_dim,
                      bias = False),
            nn.BatchNorm2d(hidden_dim),
            nn.GELU()
            # TODO: Fill this in!
        )

        """
        What's this? Remember that hidden_dim is quite large - six times the 
        in_channels. So it was nice to do the above operations in this high-dim
        space, where some patterns might be more clear. But we still want to 
        bring it back down-to-earth.

        Intuitively, you can takeaway two reasons for doing this:
        - Reduces computational cost by a lot. 6x in & out channels means 36x
          larger weights, which is crazy. We're okay with just one of input or 
          output of a convolutional layer being large when mixing channels, but 
          not both.
        
        - We also want a residual connection from the input to the output. To 
          do that without introducing another convolutional layer, we want to
          condense the # of channels back to be the same as the in_channels.
          (out_channels and in_channels are usually the same).
        """
        self.bottleneck_channels = nn.Sequential(
            nn.Conv2d(hidden_dim, out_channels, kernel_size= 1, stride= 1, padding= 0, bias= False),
            nn.BatchNorm2d(out_channels)
            # TODO: Fill this in!
        )

    def forward(self, x):
        out = self.feature_mixing(x)
        out = self.spatial_mixing(out)
        out = self.bottleneck_channels(out)

        if self.do_identity:
            return x + out
        else:
            return out

class MobileNetV2(nn.Module):
    """
    The heavy lifting is already done in InvertedBottleneck.

    Why MobileNetV2 and not V3? V2 is the foundation for V3, which uses "neural
    architecture search" to find better configurations of V2. If you understand
    V2 well, you can totally implement V3!
    """
    def __init__(self, num_classes= 7000):
        super().__init__()

        self.num_classes = num_classes

        """
        First couple of layers are special, just do them here.
        This is called the "stem". Usually, methods use it to downsample or twice.
        """
        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size= 3, stride= 2, padding= 1, bias= False),
            nn.BatchNorm2d(32),
            nn.GELU(),
            nn.Conv2d(32, 32, kernel_size= 3, stride=1 , padding= 1, groups= 32, bias= False),
            nn.BatchNorm2d(32),
            nn.GELU(),
            nn.Conv2d(32, 16, kernel_size= 1, stride= 1, padding= 0, bias= False),
            nn.BatchNorm2d(16)
            # TODO: Fill this in!
        )

        """
        Since we're just repeating InvertedResidualBlocks again and again, we
        want to specify their parameters like this.
        The four numbers in each row (a stage) are shown below.
        - Expand ratio: We talked about this in InvertedResidualBlock
        - Channels: This specifies the channel size before expansion
        - # blocks: Each stage has many blocks, how many?
        - Stride of first block: For some stages, we want to downsample. In a
          downsampling stage, we set the first block in that stage to have
          stride = 2, and the rest just have stride = 1.

        Again, note that almost every stage here is downsampling! By the time
        we get to the last stage, what is the image resolution? Can it still
        be called an image for our dataset? Think about this, and make changes
        as you want.
        """
        self.stage_cfgs = [
            # expand_ratio, channels, # blocks, stride of first block
            [6,  24, 2, 2],
            [6,  32, 3, 2],
            [6,  64, 4, 2],
            [6,  96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        # Remember that our stem left us off at 16 channels. We're going to 
        # keep updating this in_channels variable as we go
        in_channels = 16

        # Let's make the layers
        layers = []
        for curr_stage in self.stage_cfgs:
            expand_ratio, num_channels, num_blocks, stride = curr_stage
            
            for block_idx in range(num_blocks):
                out_channels = num_channels
                layers.append(InvertedResidualBlock(
                    in_channels=in_channels,
                    out_channels=out_channels,
                    # only have non-trivial stride if first block
                    stride=stride if block_idx == 0 else 1, 
                    expand_ratio=expand_ratio
                ))
                # In channels of the next block is the out_channels of the current one
                in_channels = out_channels 
            
        self.layers = nn.Sequential(*layers) # Done, save them to the class

        # Some final feature mixing
        self.final_block = nn.Sequential(
            nn.Conv2d(in_channels, 1280, kernel_size=1, padding=0, stride=1, bias=False),
            nn.BatchNorm2d(1280),
            nn.ReLU6()
        )

        # Now, we need to build the final classification layer.
        self.cls_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(1280, num_classes)
            
            # TODO: Fill this in!
            # Pool over & collapse the spatial dimensions to (1, 1)
            # Collapse the trivial (1, 1) dimensions
            # Project to our # of classes
        )

        self._initialize_weights()

    def _initialize_weights(self):
        """
        Usually, I like to use default pytorch initialization for stuff, but
        MobileNetV2 made a point of putting in some custom ones, so let's just
        use them.
        """
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

    def forward(self, x):
        out = self.stem(x)
        out = self.layers(out)
        out = self.final_block(out)
        out = self.cls_layer(out)

        return out

# Very Simple Network

In [ ]:
from torch.nn.modules.activation import ReLU
from torch.nn.modules.batchnorm import BatchNorm2d
class Network(nn.Module):
    """
    The Very Low early deadline architecture is a 4-layer CNN.
    The first Conv layer has 64 channels, kernel size 7, and stride 4.
    The next three have 128, 256, and 512 channels. Each have kernel size 3 and stride 2.
    Think about what the padding should be for each layer to not change spatial resolution.
    Each Conv layer is accompanied by a Batchnorm and ReLU layer.
    Finally, you want to average pool over the spatial dimensions to reduce them to 1 x 1.
    Then, remove (Flatten?) these trivial 1x1 dimensions away.
    Look through https://pytorch.org/docs/stable/nn.html 
    TODO: Fill out the model definition below! 

    Why does a very simple network have 4 convolutions?
    Input images are 224x224. Note that each of these convolutions downsample.
    Downsampling 2x effectively doubles the receptive field, increasing the spatial
    region each pixel extracts features from. Downsampling 32x is standard
    for most image models.

    Why does a very simple network have high channel sizes?
    Every time you downsample 2x, you do 4x less computation (at same channel size).
    To maintain the same level of computation, you 2x increase # of channels, which 
    increases computation by 4x. So, balances out to same computation.
    Another intuition is - as you downsample, you lose spatial information. Want
    to preserve some of it in the channel dimension.
    """
    def __init__(self, num_classes=7000):
        super().__init__()

        self.backbone = nn.Sequential(
            nn.Conv2d(in_channels= 3, out_channels= 64, kernel_size= 7, stride = 4, padding =  4),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels= 64, out_channels= 128, kernel_size= 3, stride = 2, padding =  1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(in_channels= 128, out_channels= 256, kernel_size= 3, stride = 2, padding =  1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(in_channels= 256, out_channels= 512, kernel_size= 3, stride = 2, padding =  1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.AvgPool2d(7),
            nn.Flatten()


            # Note that first conv is stride 4. It is (was?) standard to downsample.
            # 4x early on, as with 224x224 images, 4x4 patches are just low-level details.
            # Food for thought: Why is the first conv kernel size 7, not kernel size 3?

            # TODO: Conv group 1
            # TODO: Conv group 2
            # TODO: Conv group 3
            # TODO: Conv group 4

            # TODO: Average pool over & reduce the spatial dimensions to (1, 1)
            # TODO: Collapse (Flatten) the trivial (1, 1) dimensions
            ) 
        
        self.cls_layer = nn.Linear(512, num_classes)
    
    def forward(self, x, return_feats=False):
        """
        What is return_feats? It essentially returns the second-to-last-layer
        features of a given image. It's a "feature encoding" of the input image,
        and you can use it for the verification task. You would use the outputs
        of the final classification layer for the classification task.

        You might also find that the classification outputs are sometimes better
        for verification too - try both.
        """
        feats = self.backbone(x)
        out = self.cls_layer(feats)

        if return_feats:
            return feats
        else:
            return out

# Dataset & DataLoader

In [ ]:
"""
Transforms (data augmentation) is quite important for this task.
Go explore https://pytorch.org/vision/stable/transforms.html for more details
"""
DATA_DIR = "/content"
TRAIN_DIR = osp.join(DATA_DIR, "train_subset/train_subset") # This is a smaller subset of the data. Should change this to classification/classification/train
VAL_DIR = osp.join(DATA_DIR, "classification/classification/dev")
TEST_DIR = osp.join(DATA_DIR, "classification/classification/test")

train_transforms = [ttf.ToTensor()]
val_transforms = [ttf.ToTensor()]

train_dataset = torchvision.datasets.ImageFolder(TRAIN_DIR,
                                                 transform=ttf.Compose(train_transforms))
val_dataset = torchvision.datasets.ImageFolder(VAL_DIR,
                                               transform=ttf.Compose(val_transforms))


train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, drop_last=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        drop_last=True, num_workers=1)

# Setup everything for training

In [ ]:

batch_size = 256
lr = 0.1


model = Network()
model.cuda()

# For this homework, we're limiting you to 35 million trainable parameters, as
# outputted by this. This is to help constrain your search space and maintain
# reasonable training times & expectations
num_trainable_parameters = 0
for p in model.parameters():
    num_trainable_parameters += p.numel()
print("Number of Params: {}".format(num_trainable_parameters))

# TODO: What criterion do we use for this task?

criterion = torch.nn.CrossEntropyLoss() 
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * epochs))
# T_max is "how many times will i call scheduler.step() until it reaches 0 lr?"

# For this homework, we strongly strongly recommend using FP16 to speed up training.
# It helps more for larger models.
# Go to https://effectivemachinelearning.com/PyTorch/8._Faster_training_with_mixed_precision
# and compare "Single precision training" section with "Mixed precision training" section
scaler = torch.cuda.amp.GradScaler()

Number of Params: 5151576


# Let's train!

In [ ]:
batch_size = 256
lr = 0.1
epochs = 30

model = Network()
model.cuda()
model.load_state_dict(torch.load('/content/gdrive/MyDrive/CMU PhD semester/Sem 8/Deep Learning/HWs/HW2/part 2/First train/epoch_19.pth'))

criterion = torch.nn.CrossEntropyLoss() 
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * epochs))
# T_max is "how many times will i call scheduler.step() until it reaches 0 lr?"

# For this homework, we strongly strongly recommend using FP16 to speed up training.
# It helps more for larger models.
# Go to https://effectivemachinelearning.com/PyTorch/8._Faster_training_with_mixed_precision
# and compare "Single precision training" section with "Mixed precision training" section
scaler = torch.cuda.amp.GradScaler()

wandb.init(project="low cutoff",name='4 CNN',config={'lr':0.1,'bn':'true','num_layers':4})
wandb.watch(model, log="all")

def valid(model, val_loader):
    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
    num_correct = 0
    for i, (x, y) in enumerate(val_loader):

        x = x.cuda()
        y = y.cuda()

        with torch.no_grad():
            outputs = model(x)

        num_correct += int((torch.argmax(outputs, axis=1) == y).sum())
        batch_bar.set_postfix(acc="{:.04f}%".format(100 * num_correct / ((i + 1) * batch_size)))

        batch_bar.update()
    
    batch_bar.close()
    valid_acc = 100 * num_correct / len(val_dataset)
    # print("Validation: {:.04f}%".format(valid_acc))
    return valid_acc

for epoch in range(20, epochs +1):
    model.train()
    # Quality of life tip: leave=False and position=0 are needed to make tqdm usable in jupyter
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    num_correct = 0
    total_loss = 0

    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()

        x = x.cuda()
        y = y.cuda()
        
        # Don't be surprised - we just wrap these two lines to make it work for FP16
        with torch.cuda.amp.autocast():     
            outputs = model(x)
            loss = criterion(outputs, y)

        # Update # correct & loss as we go
        num_correct += int((torch.argmax(outputs, axis=1) == y).sum())
        total_loss += float(loss)

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / ((i + 1) * batch_size)),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        # Another couple things you need for FP16. 
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        scheduler.step() # We told scheduler T_max that we'd call step() (len(train_loader) * epochs) many times.

        batch_bar.update() # Update tqdm bar
    batch_bar.close() # You need this to close the tqdm bar

    # You can add validation per-epoch here if you would like

    
    checkpoint = { 
    'epoch': epoch,
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'lr_sched': optimizer.param_groups[0]['lr']}
    torch.save(checkpoint, '/content/gdrive/MyDrive/CMU PhD semester/Sem 8/Deep Learning/HWs/HW2/part 2/'+
               'checkpoint_'+ str(epoch)+'.pth')

    torch.save(model.state_dict(), '/content/gdrive/MyDrive/CMU PhD semester/Sem 8/Deep Learning/HWs/HW2/part 2/'+
               'epoch_'+ str(epoch)+'.pth')
    valid_acc = valid(model, val_loader)
    
    wandb.log({"Train acc": 100 * num_correct / (len(train_loader) * batch_size), "Valid acc": valid_acc, "Epoch": epoch, "learning_rate": optimizer.state_dict()['param_groups'][0]['lr']})

    print("Epoch {}/{}: Train Acc {:.04f}%, Valid Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch + 1,
        epochs, 
        100 * num_correct / (len(train_loader) * batch_size), valid_acc,
        float(total_loss / len(train_loader)),
        float(optimizer.param_groups[0]['lr'])))
wandb.finish()

Validation: 0.5629%
Epoch 21/30: Train Acc 0.7154%, Valid Acc 0.5629%, Train Loss 8.0774, Learning Rate 0.0997


Validation: 1.0457%
Epoch 22/30: Train Acc 2.7601%, Valid Acc 1.0457%, Train Loss 7.1049, Learning Rate 0.0989


Validation: 2.4286%
Epoch 23/30: Train Acc 7.2974%, Valid Acc 2.4286%, Train Loss 6.1443, Learning Rate 0.0976


Validation: 3.0829%
Epoch 24/30: Train Acc 14.0625%, Valid Acc 3.0829%, Train Loss 5.2614, Learning Rate 0.0957


Validation: 4.6286%
Epoch 25/30: Train Acc 21.9895%, Valid Acc 4.6286%, Train Loss 4.5020, Learning Rate 0.0933


...

Validation: 7.8143%
Epoch 28/30: Train Acc 47.6992%, Valid Acc 7.8143%, Train Loss 2.7755, Learning Rate 0.0835


Validation: 8.1743%
Epoch 29/30: Train Acc 56.1870%, Valid Acc 8.1743%, Train Loss 2.3254, Learning Rate 0.0794


Validation: 8.6000%
Epoch 30/30: Train Acc 64.0224%, Valid Acc 8.6000%, Train Loss 1.9341, Learning Rate 0.0750


Validation: 8.1343%
Epoch 31/30: Train Acc 71.4386%, Valid Acc 8.1343%, Train Loss 1.5966, Learning Rate 0.0703


Epoch,▁▂▂▃▄▅▅▆▇▇█
Train acc,▁▁▂▂▃▄▅▆▆▇█
Valid acc,▁▁▃▃▅▅▇▇███
learning_rate,██▇▇▆▆▅▄▃▂▁
Epoch,30
Train acc,71.43859
Valid acc,8.13429
learning_rate,0.07034


# Classification Task: Validation

# Classification Task: Submit to Kaggle

In [ ]:
class ClassificationTestSet(Dataset):
    # It's possible to load test set data using ImageFolder without making a custom class.
    # See if you can think it through!

    def __init__(self, data_dir, transforms):
        self.data_dir = data_dir
        self.transforms = transforms

        # This one-liner basically generates a sorted list of full paths to each image in data_dir
        self.img_paths = list(map(lambda fname: osp.join(self.data_dir, fname), sorted(os.listdir(self.data_dir))))

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        return self.transforms(Image.open(self.img_paths[idx]))
        # return self.transforms(Image.open(self.img_paths[idx])), self.img_paths[idx].split(sep='/')[-1]

In [ ]:
test_dataset = ClassificationTestSet(TEST_DIR, ttf.Compose(val_transforms))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                         drop_last=False, num_workers=1)

In [ ]:
# model = Network()
# model.cuda()

# model.load_state_dict(torch.load('/content/gdrive/MyDrive/CMU PhD semester/Sem 8/Deep Learning/HWs/HW2/part 2/epoch_19.pth'))


# mode.cuda()
model.eval()
batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, position=0, leave=False, desc='Test')
img_path_list = list(test_dataset.img_paths)
res = []
names = []
for i, (x) in enumerate(test_loader):
    x = x.cuda()
    # names.extend(imgp)
    with torch.no_grad():
        outputs = model(x)
    pred_y = torch.argmax(outputs, axis=1)
    res.extend(pred_y.tolist())
    
    # TODO: Finish predicting on the test set.
    

    batch_bar.update()
    
batch_bar.close()
# len(res)


In [ ]:
with open("classification_early_submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(test_dataset)):
        f.write("{},{}\n".format(str(i).zfill(6) + ".jpg", res[i]))

In [ ]:
!kaggle competitions submit -c 11-785-s22-hw2p2-classification -f classification_early_submission.csv -m/--submit1

100% 542k/542k [00:02<00:00, 209kB/s]
Successfully submitted to Face Recognition

# Verification Task: Validation

There are 6K verification dev images, but 166K "pairs" for you to compare. So, it's much more efficient to compute the features for the 6K verification images, and just compare afterwards.

This will be done by creating a dictionary mapping the image file names to the features. Then, you'll use this dictionary to compute the similarities for each pair.

In [ ]:
!ls verification/verification/dev | wc -l
!cat verification/verification/verification_dev.csv | wc -l

6000
166801


In [ ]:
class VerificationDataset(Dataset):
    def __init__(self, data_dir, transforms):
        self.data_dir = data_dir
        self.transforms = transforms

        # This one-liner basically generates a sorted list of full paths to each image in data_dir
        self.img_paths = list(map(lambda fname: osp.join(self.data_dir, fname), sorted(os.listdir(self.data_dir))))

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        # We return the image, as well as the path to that image (relative path)
        return self.transforms(Image.open(self.img_paths[idx])), osp.relpath(self.img_paths[idx], self.data_dir)

In [ ]:
val_veri_dataset = VerificationDataset(osp.join(DATA_DIR, "verification/verification/dev"),
                                       ttf.Compose(val_transforms))
val_ver_loader = torch.utils.data.DataLoader(val_veri_dataset, batch_size=batch_size, 
                                             shuffle=False, num_workers=1)

In [ ]:

model.backbone

# model = MyModel()
# model.fc2.register_forward_hook(get_activation('fc2'))
# x = torch.randn(1, 25)
# output = model(x)
# print(activation['fc2'])

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(4, 4))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (10): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU()
  (12): AvgPool2d(kernel_size=7, stride=7, padding=0)
  (13): Flatten(start_dim=1, end_dim=-1)
)

In [ ]:
from torch.nn.modules.linear import Identity
model = Network()
# model.load_state_dict(torch.load('/content/gdrive/MyDrive/CMU PhD semester/Sem 8/Deep Learning/HWs/HW2/part 2/'+
#                'epoch_'+ str(18)+'.pth', map_location=torch.device('cpu')))
# model.cuda()
model.load_state_dict(torch.load('/content/gdrive/MyDrive/CMU PhD semester/Sem 8/Deep Learning/HWs/HW2/part 2/'+
               'epoch_'+ str(30)+'.pth'))

model.cuda()
# model.load_state_dict(torch.load('/content/gdrive/MyDrive/CMU PhD semester/Sem 8/Deep Learning/HWs/HW2/part 2/epoch_19.pth'))

# model.cls_layer = Identity()

# model.backbone[2].register_backward_hook(get_activation(2))
model.eval()

feats_dict = dict()
for batch_idx, (imgs, path_names) in tqdm(enumerate(val_ver_loader), total=len(val_ver_loader), position=0, leave=False):
    
    imgs = imgs.cuda()
    with torch.no_grad():
        # Note that we return the feats here, not the final outputs
        # Feel free to try the final outputs too!
        feats = model(imgs, return_feats=False) 
    
    # TODO: Now we have features and the image path names. What to do with them?
    # Hint: use the feats_dict somehow.
    
    for i in range(len(path_names)):
        feats_dict[path_names[i]] = feats[i]
    # print(len(path_names), len(feats)) 
    # break




In [ ]:
# What does this dict look like?
print(list(feats_dict.items())[0])

In [ ]:
# We use cosine similarity between feature embeddings.
# TODO: Find the relevant function in pytorch and read its documentation.
similarity_metric = nn.CosineSimilarity(dim=0) 

val_veri_csv = osp.join(DATA_DIR, "verification/verification/verification_dev.csv")


# Now, loop through the csv and compare each pair, getting the similarity between them
pred_similarities = []
gt_similarities = []
for line in tqdm(open(val_veri_csv).read().splitlines()[1:], position=0, leave=False): # skip header
    img_path1, img_path2, gt = line.split(",")
    
    # TODO: Use the similarity metric
    # How to use these img_paths? What to do with the features?
    similarity = similarity_metric(feats_dict[img_path1.split("/")[-1]], feats_dict[img_path2.split("/")[-1]])
    pred_similarities.append(float(similarity))
    gt_similarities.append(int(gt))

pred_similarities = np.array(pred_similarities)
gt_similarities = np.array(gt_similarities)

# pred_similarities = pred_similarities
# gt_similarities = gt_similarities.numpy()

print("AUC:", roc_auc_score(gt_similarities, pred_similarities))

AUC: 0.8256664958969158


# Verification Task: Submit to Kaggle

In [ ]:
test_veri_dataset = VerificationDataset(osp.join(DATA_DIR, "verification/verification/test"),
                                        ttf.Compose(val_transforms))
test_ver_loader = torch.utils.data.DataLoader(test_veri_dataset, batch_size=batch_size, 
                                              shuffle=False, num_workers=1)

In [ ]:
model.eval()

feats_dict = dict()
for batch_idx, (imgs, path_names) in tqdm(enumerate(test_ver_loader), total=len(test_ver_loader), position=0, leave=False):
    imgs = imgs.cuda()
    with torch.no_grad():
        # Note that we return the feats here, not the final outputs
        # Feel free to try the final outputs too!
        feats = model(imgs, return_feats=False) 
    
    # TODO: Now we have features and the image path names. What to do with them?
    # Hint: use the feats_dict somehow.
    
    for i in range(len(path_names)):
        feats_dict[path_names[i]] = feats[i]
    
    # TODO: Now we have features and the image path names. What to do with them?
    # Hint: use the feats_dict somehow.

In [ ]:
# We use cosine similarity between feature embeddings.
# TODO: Find the relevant function in pytorch and read its documentation.
# similarity_metric = 
val_veri_csv = osp.join(DATA_DIR, "verification/verification/verification_test.csv")


# Now, loop through the csv and compare each pair, getting the similarity between them
pred_similarities = []
for line in tqdm(open(val_veri_csv).read().splitlines()[1:], position=0, leave=False): # skip header
    img_path1, img_path2 = line.split(",")

    similarity = similarity_metric(feats_dict[img_path1.split("/")[-1]], feats_dict[img_path2.split("/")[-1]])
    pred_similarities.append(similarity)
    # gt_similarities.append(int(gt))

# pred_similarities = np.array(pred_similarities)
# gt_similarities = np.array(gt_similarities)


In [ ]:
with open("verification_early_submission.csv", "w+") as f:
    f.write("id,match\n")
    for i in range(len(pred_similarities)):
        f.write("{},{}\n".format(i, pred_similarities[i]))

In [ ]:
!kaggle competitions submit -c 11-785-s22-hw2p2-verification -f verification_early_submission.csv -m/--first

100% 16.8M/16.8M [00:01<00:00, 9.18MB/s]
Successfully submitted to Face Verification

# Extras

In [ ]:
# If you keep re-initializing your model in Colab, can run out of GPU memory, need to restart.
# These three lines can help that - run this before you re-initialize your model

del model
torch.cuda.empty_cache()
!nvidia-smi

Fri Feb 25 17:15:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    77W / 149W |   1378MiB / 11441MiB |     35%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
a = 2